---
format:
  html:
    embed-resources: true
    fig-width: 9
    fig-height: 6
jupyter: python3
code-fold: true
code-overflow: wrap
---

In [54]:
import numpy as np
from scipy.linalg import solve, lu, qr, cholesky, eig, svd
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import cg  # Conjugate Gradient

# Computational approaches

## NumPy and SciPy linear algebra solvers

### NumPy

NumPy provides basic linear algebra routines through $\texttt{numpy.linalg}$. These functions internally use **LAPACK** and **BLAS** for efficiency.


1. **Solving a system of linear equations**  
   Given the system:

   $$ Ax = b $$

In [55]:
A = np.array([[3, 2], [1, 4]])
B = np.array([5, 6])
x = np.linalg.solve(A, B)  # Solves Ax = B
print(x)

[0.8 1.3]


This function uses an efficient algorithm (Gaussian elimination with partial pivoting) to find x. Refer to [this link](https://numpy.org/doc/2.2/reference/generated/numpy.linalg.solve.html) for more information.

2. **Computing the inverse of a matrix**
  $$ A^{-1} $$

In [56]:
 
   A_inv = np.linalg.inv(A) # Inverse of A
   x = np.dot(A_inv, B) # Solves Ax = B
   print(x)

[0.8 1.3]


Note that the matrix must non-singular $(\det(A) \neq 0)$  for this to work. Click the [link](https://numpy.org/doc/2.2/reference/generated/numpy.linalg.inv.html) for more information.

3. **Computing the determinant**<br>
The determinant of a square matrix A is a scalar value that provides important information about the matrix. It is denoted as:
   $$ \det(A) $$

In [57]:
det_A = np.linalg.det(A) # Determinant of A
# Check if the determinant is zero
if det_A == 0:
    print("The system has no unique solution (determinant is zero).")
else:
    x = np.linalg.solve(A, B) # Solves Ax = B
    print(x)

[0.8 1.3]


A nonzero determinant means A is invertible, while det(A) = 0 means A is singular (not invertible).

4. **Eigenvalues and Eigenvectors**

For a square matrix \( $A$ \), the eigenvalues (\( $\lambda$ \)) and corresponding eigenvectors (\( $v$ \)) satisfy the equation:

$$
A \cdot v = \lambda \cdot v
$$

where:

- \( $\lambda$ \) (lambda) is a **scalar** (eigenvalue),
- \( $v$ \) is a **nonzero vector** (eigenvector).

In [58]:
# Eigenvalues and eigenvectors
eigenvalues, eigenvectors  = np.linalg.eig(A) # compute eigenvalues and eigenvectors

# Diagonal matrix of eigenvalues
D = np.diag(eigenvalues)

# Compute P inverse
P_inv = np.linalg.inv(eigenvectors)

# Transform B into the eigenbasis
B_prime = P_inv @ B

# Solve for Y in DY = B'
Y = np.linalg.solve(D, B_prime)

# Compute the final solution X = P Y
X = eigenvectors @ Y

# Print the result
print("Solution X:", X)

Solution X: [0.8 1.3]


ELI5: Eigenvectors "point" in the same direction as most of your data, and eigenvalues tell you how strongly your data points that direction. 

5. **Singular Value Decomposition (SVD)**

Singular Value Decomposition (SVD) is a method to break down a complex matrix into simpler components. It is similar to taking a blurry image and separating it into clear building blocks.

### SVD Formula
For any matrix \( $A$ \), SVD expresses it as:

$$ A = U \Sigma V^T $$

Where:
- **\( $U$ \)** (Left singular vectors) → Contains important patterns from the original data.
- **\( $\Sigma$ \)** (Singular values) → A diagonal matrix with values indicating the importance of each pattern.
- **\( $V^{T}$ \)** (Right singular vectors) → Contains another set of patterns that, when combined with \( $\Sigma$ \), recreate the original data.

Analogy:<br>
Imagine you have a large playlist of songs. SVD breaks it down into:
- **\( $U$\)** → The general themes of music (e.g., rock, jazz, pop).
- **\( $\Sigma$ \)** → How strongly each theme appears in the playlist.
- **\( $V^{T}$ \)** → The details of each song, arranged by themes.

In [59]:
# Compute Singular Value Decomposition (SVD)
U, S, V = np.linalg.svd(A)

# Convert S into a diagonal matrix
S_diag = np.diag(S)

# Compute the pseudo-inverse of S
S_inv = np.linalg.inv(S_diag)

# Compute the pseudo-inverse of A using SVD
A_pinv = V.T @ S_inv @ U.T # @ means matrix multiplication and .T means transpose

# Solve for X using the pseudo-inverse
X = A_pinv @ B

# Print the result
print("Solution X:", X)

Solution X: [0.8 1.3]


### SciPy

SciPy extends NumPy's linear algebra capabilities with additional **matrix decompositions, advanced solvers, and performance optimizations**.
Key Advantages of SciPy over NumPy:<br>
- Uses **LAPACK** and **ATLAS**, but with extra options for optimization.
- Supports sparse matrices via \texttt{scipy.sparse.linalg}.
- Includes additional decomposition methods.

1. **Solving a linear system ($\texttt{scipy.linalg.solve}$)**  
   Same as $\texttt{numpy.linalg.solve}$ but often faster and more stable.


In [60]:
x = solve(A, B)  # More efficient than np.linalg.solve for large matrices
print(x)

[0.8 1.3]


2. **LU Decomposition**  

   $$ PA = LU $$

In [63]:
P, L, U = lu(A)  # P: Permutation, L: Lower triangular, U: Upper triangular

# Compute Pb
Pb = np.dot(P, B)

# Solve Ly = Pb using forward substitution
y = np.linalg.solve(L, Pb)

# Solve Ux = y using backward substitution
x = np.linalg.solve(U, y)
x

array([0.8, 1.3])

3. **QR Decomposition**  

   $$ A = QR $$

In [64]:
Q, R = qr(A)
# Compute Q^T B
QTB = np.dot(Q.T, B)

# Solve Rx = Q^T B using backward substitution
x = np.linalg.solve(R, QTB)
x

array([0.8, 1.3])

4. **Cholesky Decomposition (for positive definite matrices)**  

   $$ A = LL^T $$

In [66]:
L = cholesky(A, lower=True)
# Solve Ly = B using forward substitution
y = np.linalg.solve(L, B)

# Solve L^T x = y using backward substitution
x = np.linalg.solve(L.T, y)

x

array([1.27272727, 1.18181818])

The answer is different, issue coming from the fact that Cholesky decomposition only works for symmetric positive definite (SPD) matrices and the example above is **NOT** symmetric.

5. **Eigenvalue Problems ($\texttt{scipy.linalg.eig}$)**

In [68]:
eigvals, eigvecs = eig(A)

# Compute V^(-1) * B
V_inv_B = np.linalg.solve(eigvecs, B)

# Solve D * y = V_inv_B (since D is diagonal, just divide)
y = V_inv_B / eigvals

# Compute final solution x = V * y
x = np.dot(eigvecs, y)
x

array([0.8+0.j, 1.3+0.j])

6. **Singular Value Decomposition (SVD)**  

   $$ A = U S V^T $$

In [47]:
U, S, V = svd(A)

### Sparse Linear Algebra ($\texttt{scipy.sparse.linalg}$)
For large, sparse matrices, SciPy provides \texttt{scipy.sparse.linalg}, which is optimized for efficiency.

1. **Solving a sparse linear system**  

   $$ Ax = b $$


In [49]:
A_sparse = csc_matrix(A)  # Convert A to sparse format
x = spsolve(A_sparse, B)

2. **Iterative solvers (Conjugate Gradient, GMRES)**  

In [ ]:
x, info = cg(A_sparse, B)

## Iterative solution techniques